In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from classifier import *
import features as fe 
import utils as ut

In [ ]:
X_train_face_img, X_train_nonface_img, X_test_img, y_train_face, y_train_nonface, y_test = ut.get_test_train_data()

### Testing AdaBoost

In [ ]:
X_train_img = np.concatenate((X_train_face_img, X_train_nonface_img))
y_train = np.concatenate((y_train_face, y_train_nonface))
X_train_ada, y_train_ada = ut.random_subset(X_train_img, y_train, 100)
X_test_ada, y_test_ada = ut.random_subset(X_test_img, y_test, 100)

In [ ]:
rect = fe.get_rectanges(19, 19)
no_rect = fe.get_no_rectangles(19, 19)
X_train_ada_fe = fe.par_feature_extraction_images(X_train_ada, rect, no_rect)
X_test_ada_fe = fe.par_feature_extraction_images(X_test_ada, rect, no_rect)

In [ ]:
from classifier import *

In [ ]:
a = AdaBoostClassifier()
a.fit(X_train_ada_fe, y_train_ada, 2)
accuracy = a.score(X_test_ada_fe, y_test_ada)
print(accuracy)

In [ ]:
iters = list(range(1, 10))
accuracies = []
a = AdaBoostClassifier()
for iter in iters:
    a.fit(X_train_ada_fe, y_train_ada, iter)
    accuracy = a.score(X_test_ada_fe, y_test_ada)
    accuracies.append(accuracy)

plt.xlabel("iterations")
plt.ylabel("accuraties")
plt.plot(iters, accuracies)
plt.show()

In [7]:
print(a.threshold*0.98)
print(a.false_positive_rate(X_test_ada_fe, y_test_ada))
print(a.detection_rate(X_test_ada_fe, y_test_ada))

12.513921067792554
0.04040404040404041
0.0


In [8]:
# split the training data into 10 folds
X_train_folds = np.array_split(X_train_ada_fe, 10)
y_train_folds = np.array_split(y_train_ada, 10)


In [47]:
f = 0.9
d = 0.9
F_target = 0.001
Threshold_retention = 0.99
Strong_Classifiers = []

def Cascade_Classifier(X_test,y_test, Strong_Classifiers):
    y_pred = np.zeros(len(X_test))
    no_of_negatives = np.sum(y_test == 0)
    total_test_y = len(y_test)
    for i in range(len(Strong_Classifiers)):
        print(y_pred.shape, X_test.shape,y_test.shape)
        y_pred = Strong_Classifiers[i].predict(X_test)
        X_test = X_test[y_pred == 1]
        y_test = y_test[y_pred == 1]
        y_pred = y_pred[y_pred == 1]
        if len(X_test) == 0:
            break
    false_positive_rate = np.sum((y_pred == 1) & (y_test == 0))/no_of_negatives
    detection_rate = np.sum((y_pred == 1)& (y_test == 1))/total_test_y
    return y_pred, false_positive_rate, detection_rate


def get_avg_fpr_dtr(Strong_Classifiers,a,X_train_folds, y_train_folds, no_of_folds,no_of_features):
    false_positive_rate = 0
    detection_rate = 0
    indices = np.random.choice(10, no_of_folds, replace=False)
    X_test = []
    y_test = []
    for i in indices:
        X_test.append(X_train_folds[i])
        y_test.append(y_train_folds[i])

    X_test = np.concatenate(X_test)
    y_test = np.concatenate(y_test)
    X_train = np.concatenate([X_train_folds[i] for i in range(10) if i not in indices])
    y_train = np.concatenate([y_train_folds[i] for i in range(10) if i not in indices])
    print("shapes before: " ,X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    for i in range(len(Strong_Classifiers)):
        y_pred = Strong_Classifiers[i].predict(X_train)
        X_train = X_train[y_pred == 1]
        y_train = y_train[y_pred == 1]
        if len(X_train) == 0:
            break
    # fit a new classifier on the remaining data
    print("shapes after: " ,X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    a.fit(X_train, y_train, no_of_features)
    # add the classifier to the list of strong classifiers
    Strong_Classifiers.append(a)    
    pred_values,false_positive_rate, detection_rate = Cascade_Classifier(X_test,y_test, Strong_Classifiers)
    return false_positive_rate, detection_rate, Strong_Classifiers[-1], X_test, y_test,Strong_Classifiers

def Train_Cascade(f,d,F_target,Threshold_retention):    
    F = [0.6]
    D = [0.6]
    F_new = F[-1]
    First_Strong_Classifier = AdaBoostClassifier()    
    Strong_Classifiers = []
    while F_new > F_target:
        a = AdaBoostClassifier()
        F_new = F[-1]
        no_of_features = 0        
        while F_new > f*F[-1]:
            no_of_features += 1       
            fpr, dtr, a,X_test_returned,y_test_returned,Strong_Classifiers = get_avg_fpr_dtr(Strong_Classifiers,a,X_train_folds, y_train_folds, 3,no_of_features)
            F_new = fpr
            D_new = dtr
            print("passed the first while loop")
            # decrease the threshold for the ith strong classifier in the cascade
            while(True):            
                Strong_Classifiers[-1].threshold = Strong_Classifiers[-1].threshold*Threshold_retention
                print(Strong_Classifiers[-1].threshold)
                a = Strong_Classifiers[-1]
                print(a.threshold)
                y_preds,fpr,dtr = Cascade_Classifier(X_test_returned,y_test_returned, Strong_Classifiers)
                F_new = fpr
                D_new = dtr
                print("in the second while loop and the threshold is", a.threshold," ", F_new, " ", D_new," ",d*D[-1]," ",no_of_features)
                if D_new > d*D[-1]:          
                    break      
            # remove the last strong classifier from the cascade
            Strong_Classifiers.pop()


        print(F_new)          
        F.append(F_new)
        D.append(D_new)
        Strong_Classifiers.append(a)     
    return Strong_Classifiers

Strong_Classifiers = np.array(Train_Cascade(f,d,F_target,Threshold_retention))
print("the length of the strong classifiers is", len(Strong_Classifiers))


shapes before:  (70, 51705) (70,) (30, 51705) (30,)
shapes after:  (70, 51705) (70,) (30, 51705) (30,)
(30,) (30, 51705) (30,)
passed the first while loop
1.127247306079829
1.127247306079829
(30,) (30, 51705) (30,)
in the second while loop and the threshold is 1.127247306079829   0.09523809523809523   0.26666666666666666   0.54   1
1.1159748330190307
1.1159748330190307
(30,) (30, 51705) (30,)
in the second while loop and the threshold is 1.1159748330190307   0.09523809523809523   0.26666666666666666   0.54   1
1.1048150846888403
1.1048150846888403
(30,) (30, 51705) (30,)
in the second while loop and the threshold is 1.1048150846888403   0.09523809523809523   0.26666666666666666   0.54   1
1.0937669338419518
1.0937669338419518
(30,) (30, 51705) (30,)
in the second while loop and the threshold is 1.0937669338419518   0.09523809523809523   0.26666666666666666   0.54   1
1.0828292645035322
1.0828292645035322
(30,) (30, 51705) (30,)
in the second while loop and the threshold is 1.0828292645

KeyboardInterrupt: 

In [29]:
y_pred, false_positive_rate, detection_rate = Cascade_Classifier(X_test_ada_fe,y_test_ada, Strong_Classifiers)
print("the accuracy of the cascade classifier is", np.sum(y_pred == y_test_ada)/len(y_test_ada))

the accuracy of the cascade classifier is 0.99
0
